In [ ]:
import pandas as pd

import plotly

import plotly.graph_objs as go


import plotly.offline as offline

from plotly.graph_objs import *

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
import pandas as pd
data_xls = pd.read_excel('../data/Folkhalsomyndigheten_Covid19.xlsx', 'Antal per dag region', index_col=1)
data_xls.to_csv('../data/Sweden_covid.csv', encoding='utf-8')
df_sweden = pd.read_csv('../data/Sweden_covid.csv')

In [ ]:
for region in df_sweden.drop(["Statistikdatum"], axis = 1).columns:
    df_sweden[region] = df_sweden[region].cumsum()

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/sweden-counties.geojson') as response:
    counties = json.load(response)

sweden_counties = []
for x in range(len(counties['features'])):
    sweden_counties.append(counties['features'][x]['properties']['name'])

In [ ]:
data_slider = []

scl = [[0.0, '#ffffff'],[0.2, '#ff9999'],[0.4, '#ff4d4d'], \
       [0.6, '#ff1a1a'],[0.8, '#cc0000'],[1.0, '#4d0000']] # reds

for date in df_sweden["Statistikdatum"].unique():
    
    sweden_one_date = df_sweden[df_sweden["Statistikdatum"] == date].drop(["Statistikdatum", "Totalt_antal_fall"], axis = 1).T
    
    new_df = pd.DataFrame({"county": sweden_one_date.index,
                           "infected": sweden_one_date.T.values.tolist()[0]})
    
    new_df.loc[new_df["county"] == "Jämtland_Härjedalen", "county"] = "Jämtland"
    new_df.loc[new_df["county"] == "Sörmland", "county"] = "Södermanland"
    new_df.loc[new_df["county"] == "Västra_Götaland", "county"] = "Västra Götaland"
    
    data_one_date = dict(
                        type='choropleth', 
                        geojson = counties, 
                        colorscale = scl, 
                        locations = new_df["county"],
                        featureidkey = 'properties.name',
                        z=new_df["infected"],
                        locationmode='geojson-id',
                        text = new_df["county"],
                        zmin = 0,
                        zmax = 8100,
        
                        marker = dict(     # for the lines separating states
                        line = dict (color = 'rgb(0,0,0)', width = 0.5) ),
                        
                        colorbar = dict(title = "Number of infected people in a given day") 
                        )

    data_slider.append(data_one_date)  # I add the dictionary to the list of dictionaries for the slider


In [ ]:
steps = []

for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label= "Day {}".format(i))
    step['args'][1][i] = True
    steps.append(step)


##  I create the 'sliders' object from the 'steps' 

sliders = [dict(active=0, pad={"t": 1}, steps=steps, ticklen = 10, len = 750, lenmode = 'pixels')]

In [ ]:
layout = dict(title = "Sweden counties COVID19",
              width = 850, 
              height = 680, 
              hovermode = "closest",
              geo=dict(scope='europe',
                       showlakes = True, 
                       lakecolor = 'rgb(73, 216, 230)',
                        lataxis = {"range": [55.0, 75.0]}, 
                        lonaxis = {"range": [10, 30.0]}, 
                        showland = False
                      ),
              sliders=sliders)

In [ ]:
fig = dict(data=data_slider, layout=layout) 

# to plot in the notebook

plotly.offline.iplot(fig)

In [ ]:
offline.plot(fig, auto_open=False, image_width=2000, image_height=1000, 
              filename='../figures/sweden_covid.html', validate=True)
